## 📦 1. Installation et Imports

In [ ]:
# Installation des packages
!pip install -q transformers datasets

print("✅ Installation terminée!")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from datetime import datetime
import json
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow import keras

# Transformers
from transformers import (
    BertTokenizer,
    TFBertModel,
    TFBertForSequenceClassification
)

from sklearn.metrics import (
    precision_score, recall_score, f1_score,
    hamming_loss, accuracy_score, classification_report
)

# Configuration
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Seed pour reproductibilité
np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU disponible: {len(tf.config.list_physical_devices('GPU')) > 0}")

# Vérifier la mémoire GPU
if tf.config.list_physical_devices('GPU'):
    gpu = tf.config.list_physical_devices('GPU')[0]
    print(f"GPU: {gpu}")
    
print("✅ Imports réussis!")

## 📂 2. Création des Dossiers et Montage Google Drive (Optionnel)

In [ ]:
# Créer les dossiers nécessaires
folders = [
    'data/processed',
    'models/bert',
    'results/figures',
    'results/metrics'
]

for folder in folders:
    Path(folder).mkdir(parents=True, exist_ok=True)
    
print("✅ Dossiers créés!")

# Optionnel: Monter Google Drive pour sauvegarder les résultats
MOUNT_DRIVE = False  # Mettre à True pour monter Google Drive

if MOUNT_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive')
    print("✅ Google Drive monté!")

## 📥 3. Chargement des Données Brutes

⚠️ **Important**: BERT nécessite son propre tokenizer, donc on charge les textes bruts (pas les séquences pré-tokenizées).

In [ ]:
# Monter Google Drive
from google.colab import drive
drive.mount('/content/drive')

print("Chargement des données brutes depuis Google Drive...\n")

# Définir le chemin vers les données dans Drive
DATA_PATH = '/content/drive/MyDrive/emotion_detection_project/processed'

# Charger les textes bruts
with open(f'{DATA_PATH}/texts_train.pkl', 'rb') as f:
    texts_train = pickle.load(f)
    
with open(f'{DATA_PATH}/texts_val.pkl', 'rb') as f:
    texts_val = pickle.load(f)
    
with open(f'{DATA_PATH}/texts_test.pkl', 'rb') as f:
    texts_test = pickle.load(f)
    
print("✅ Textes chargés")

# Charger les labels
y_train = np.load(f'{DATA_PATH}/y_train.npy')
y_val = np.load(f'{DATA_PATH}/y_val.npy')
y_test = np.load(f'{DATA_PATH}/y_test.npy')
print("✅ Labels chargés")

# Charger les métadonnées
with open(f'{DATA_PATH}/metadata.pkl', 'rb') as f:
    metadata = pickle.load(f)

NUM_CLASSES = metadata['num_classes']
EMOTION_LABELS = metadata['emotion_labels']

print(f"\n📊 Statistiques:")
print(f"  Nombre de classes: {NUM_CLASSES}")
print(f"  Train: {len(texts_train):,} | Val: {len(texts_val):,} | Test: {len(texts_test):,}")

## 🔤 4. Tokenization BERT

BERT utilise WordPiece tokenization et nécessite des tokens spéciaux [CLS] et [SEP].

In [ ]:
print("🔤 Chargement du tokenizer BERT...\n")

# Charger le tokenizer BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Configuration
MAX_LENGTH = 128  # BERT peut gérer jusqu'à 512, mais 128 est suffisant ici

print(f"✅ Tokenizer chargé: bert-base-uncased")
print(f"  Vocab size: {tokenizer.vocab_size:,}")
print(f"  Max length: {MAX_LENGTH}")

In [ ]:
print("\n🔄 Tokenization des textes...\n")

# Tokenizer les textes
def tokenize_texts(texts, tokenizer, max_length=128):
    """Tokenizer les textes avec BERT tokenizer"""
    return tokenizer(
        texts,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors='tf'
    )

# Tokenizer train
print("Tokenization train...")
train_encodings = tokenize_texts(texts_train, tokenizer, MAX_LENGTH)

# Tokenizer val
print("Tokenization val...")
val_encodings = tokenize_texts(texts_val, tokenizer, MAX_LENGTH)

# Tokenizer test
print("Tokenization test...")
test_encodings = tokenize_texts(texts_test, tokenizer, MAX_LENGTH)

print("\n✅ Tokenization terminée!")
print(f"  Shape input_ids: {train_encodings['input_ids'].shape}")
print(f"  Shape attention_mask: {train_encodings['attention_mask'].shape}")

## 🏗️ 5. Construction du Modèle BERT

Utilisation de TFBertModel + couche de classification personnalisée.

In [ ]:
def create_bert_model(num_classes, learning_rate=2e-5):
    """
    Crée un modèle BERT pour la classification multi-label
    
    Architecture:
    - BERT base (pré-entraîné)
    - Dropout
    - Dense(num_classes, sigmoid)
    """
    
    # Charger BERT pré-entraîné
    bert = TFBertModel.from_pretrained('bert-base-uncased')
    
    # Inputs
    input_ids = keras.layers.Input(shape=(MAX_LENGTH,), dtype=tf.int32, name='input_ids')
    attention_mask = keras.layers.Input(shape=(MAX_LENGTH,), dtype=tf.int32, name='attention_mask')
    
    # BERT encoding
    bert_output = bert(input_ids, attention_mask=attention_mask)
    
    # Prendre la sortie [CLS] (premier token)
    cls_output = bert_output.last_hidden_state[:, 0, :]
    
    # Classification head
    x = keras.layers.Dropout(0.3)(cls_output)
    outputs = keras.layers.Dense(num_classes, activation='sigmoid', name='output')(x)
    
    # Créer le modèle
    model = keras.Model(
        inputs=[input_ids, attention_mask],
        outputs=outputs,
        name='BERT_Emotion_Classifier'
    )
    
    # Compiler
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=[
            'accuracy',
            keras.metrics.Precision(name='precision'),
            keras.metrics.Recall(name='recall'),
            keras.metrics.AUC(name='auc')
        ]
    )
    
    return model

print("🏗️ Construction du modèle BERT...\n")
model = create_bert_model(NUM_CLASSES, learning_rate=2e-5)

# Afficher l'architecture
model.summary()

# Compter les paramètres
total_params = model.count_params()
print(f"\n📊 Total paramètres: {total_params:,}")
print(f"⚠️ Note: BERT base contient ~110M de paramètres!")

## 🎯 6. Configuration des Callbacks

In [ ]:
# Créer les callbacks
callbacks = [
    # Early stopping: arrêter si pas d'amélioration après 3 epochs
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=3,
        restore_best_weights=True,
        verbose=1
    ),
    
    # Model checkpoint: sauvegarder le meilleur modèle
    keras.callbacks.ModelCheckpoint(
        filepath='models/bert/best_model',
        monitor='val_loss',
        save_best_only=True,
        save_format='tf',
        verbose=1
    ),
    
    # Reduce learning rate: réduire le LR si plateau
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=2,
        min_lr=1e-7,
        verbose=1
    )
]

print("✅ Callbacks configurés!")

## 🚀 7. Entraînement du Modèle

⏱️ **Temps estimé**: 50-60 minutes sur GPU  
⚠️ **Attention**: Très lent sur CPU (>8h), utiliser GPU si possible!

In [ ]:
print("🚀 Début de l'entraînement BERT...\n")
print("⚠️ Cela peut prendre du temps (50-60 min sur GPU)...\n")

start_time = datetime.now()

# Entraîner le modèle
history = model.fit(
    {
        'input_ids': train_encodings['input_ids'],
        'attention_mask': train_encodings['attention_mask']
    },
    y_train,
    validation_data=(
        {
            'input_ids': val_encodings['input_ids'],
            'attention_mask': val_encodings['attention_mask']
        },
        y_val
    ),
    epochs=5,  # 5 epochs suffisent généralement pour BERT
    batch_size=16,  # Petit batch size pour GPU memory
    callbacks=callbacks,
    verbose=1
)

end_time = datetime.now()
training_time = (end_time - start_time).total_seconds()

print(f"\n✅ Entraînement terminé!")
print(f"⏱️ Temps d'entraînement: {training_time/60:.2f} minutes")

## 📊 8. Visualisation de l'Entraînement

In [ ]:
# Créer une figure avec 4 sous-graphiques
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Loss
axes[0, 0].plot(history.history['loss'], label='Train Loss', linewidth=2, marker='o')
axes[0, 0].plot(history.history['val_loss'], label='Val Loss', linewidth=2, marker='o')
axes[0, 0].set_title('Loss Evolution', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Accuracy
axes[0, 1].plot(history.history['accuracy'], label='Train Accuracy', linewidth=2, marker='o')
axes[0, 1].plot(history.history['val_accuracy'], label='Val Accuracy', linewidth=2, marker='o')
axes[0, 1].set_title('Accuracy Evolution', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Accuracy')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Precision
axes[1, 0].plot(history.history['precision'], label='Train Precision', linewidth=2, marker='o')
axes[1, 0].plot(history.history['val_precision'], label='Val Precision', linewidth=2, marker='o')
axes[1, 0].set_title('Precision Evolution', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Precision')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Recall
axes[1, 1].plot(history.history['recall'], label='Train Recall', linewidth=2, marker='o')
axes[1, 1].plot(history.history['val_recall'], label='Val Recall', linewidth=2, marker='o')
axes[1, 1].set_title('Recall Evolution', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('Recall')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('results/figures/bert_training.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Graphiques sauvegardés!")

## 🎯 9. Évaluation sur l'Ensemble de Test

In [ ]:
print("🎯 Évaluation sur l'ensemble de test...\n")

# Prédictions
y_pred_proba = model.predict(
    {
        'input_ids': test_encodings['input_ids'],
        'attention_mask': test_encodings['attention_mask']
    },
    batch_size=32,
    verbose=1
)
y_pred = (y_pred_proba > 0.5).astype(int)

# Calculer les métriques
precision_micro = precision_score(y_test, y_pred, average='micro', zero_division=0)
precision_macro = precision_score(y_test, y_pred, average='macro', zero_division=0)

recall_micro = recall_score(y_test, y_pred, average='micro', zero_division=0)
recall_macro = recall_score(y_test, y_pred, average='macro', zero_division=0)

f1_micro = f1_score(y_test, y_pred, average='micro', zero_division=0)
f1_macro = f1_score(y_test, y_pred, average='macro', zero_division=0)

hamming = hamming_loss(y_test, y_pred)
subset_acc = accuracy_score(y_test, y_pred)

# Afficher les résultats
print("="*60)
print("📊 RÉSULTATS SUR L'ENSEMBLE DE TEST")
print("="*60)
print(f"\n🎯 Métriques Globales:")
print(f"  Precision (micro): {precision_micro:.4f}")
print(f"  Precision (macro): {precision_macro:.4f}")
print(f"  Recall (micro):    {recall_micro:.4f}")
print(f"  Recall (macro):    {recall_macro:.4f}")
print(f"  F1-Score (micro):  {f1_micro:.4f}")
print(f"  F1-Score (macro):  {f1_macro:.4f}")
print(f"  Hamming Loss:      {hamming:.4f}")
print(f"  Subset Accuracy:   {subset_acc:.4f}")
print("="*60)

## 📈 10. Métriques par Classe

In [ ]:
# Calculer les métriques par classe
precision_per_class = precision_score(y_test, y_pred, average=None, zero_division=0)
recall_per_class = recall_score(y_test, y_pred, average=None, zero_division=0)
f1_per_class = f1_score(y_test, y_pred, average=None, zero_division=0)

# Créer un DataFrame
metrics_df = pd.DataFrame({
    'Emotion': EMOTION_LABELS,
    'Precision': precision_per_class,
    'Recall': recall_per_class,
    'F1-Score': f1_per_class,
    'Support': y_test.sum(axis=0)
})

# Trier par F1-Score
metrics_df = metrics_df.sort_values('F1-Score', ascending=False)

print("\n📊 TOP 10 Émotions (par F1-Score):")
print(metrics_df.head(10).to_string(index=False))

print("\n📊 BOTTOM 10 Émotions (par F1-Score):")
print(metrics_df.tail(10).to_string(index=False))

# Sauvegarder
metrics_df.to_csv('results/metrics/bert_per_class.csv', index=False)
print("\n✅ Métriques par classe sauvegardées!")

## 📊 11. Visualisation des Métriques par Classe

In [ ]:
# Visualiser les 15 meilleures et 15 pires émotions
fig, axes = plt.subplots(1, 2, figsize=(18, 8))

# Top 15
top_15 = metrics_df.head(15).sort_values('F1-Score')
axes[0].barh(top_15['Emotion'], top_15['F1-Score'], color='green', alpha=0.7)
axes[0].set_xlabel('F1-Score', fontsize=12)
axes[0].set_title('Top 15 Émotions (Meilleures F1-Scores)', fontsize=14, fontweight='bold')
axes[0].grid(axis='x', alpha=0.3)

# Bottom 15
bottom_15 = metrics_df.tail(15).sort_values('F1-Score')
axes[1].barh(bottom_15['Emotion'], bottom_15['F1-Score'], color='red', alpha=0.7)
axes[1].set_xlabel('F1-Score', fontsize=12)
axes[1].set_title('Bottom 15 Émotions (Pires F1-Scores)', fontsize=14, fontweight='bold')
axes[1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.savefig('results/figures/bert_per_class.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Graphiques sauvegardés!")

## 💾 12. Sauvegarde des Résultats Complets

In [ ]:
# Sauvegarder les résultats JSON
results = {
    'model_name': 'BERT',
    'model_version': 'bert-base-uncased',
    'timestamp': datetime.now().isoformat(),
    'training_time_minutes': training_time / 60,
    'total_params': int(total_params),
    'max_length': MAX_LENGTH,
    'metrics': {
        'precision_micro': float(precision_micro),
        'precision_macro': float(precision_macro),
        'recall_micro': float(recall_micro),
        'recall_macro': float(recall_macro),
        'f1_micro': float(f1_micro),
        'f1_macro': float(f1_macro),
        'hamming_loss': float(hamming),
        'subset_accuracy': float(subset_acc)
    },
    'best_epoch': int(np.argmin(history.history['val_loss'])) + 1,
    'best_val_loss': float(min(history.history['val_loss']))
}

with open('results/metrics/bert_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print("✅ Résultats sauvegardés dans results/metrics/bert_results.json")

# Sauvegarder l'historique
history_df = pd.DataFrame(history.history)
history_df.to_csv('results/metrics/bert_history.csv', index=False)
print("✅ Historique sauvegardé!")

# Sauvegarder les prédictions
np.save('results/metrics/bert_predictions.npy', y_pred_proba)
print("✅ Prédictions sauvegardées!")

print("\n" + "="*60)
print("🎉 NOTEBOOK BERT TERMINÉ!")
print("="*60)